In [2]:
# Fama & French 3 factors residual variance
# Note: Please use the latest version of pandas, this version should support returning to pd.Series after rolling
# To get a faster speed, we split the big dataframe into small ones
# Then using different process to calculate the variance
# We use 20 process to calculate variance, you can change the number of process according to your CPU situation
# You can use the following code to check your CPU situation
# import multiprocessing
# multiprocessing.cpu_count()

import pandas as pd
import numpy as np
import datetime as dt
import wrds
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import datetime
import pickle as pkl
import statsmodels.api as sm
import pyarrow.feather as feather
import multiprocessing as mp

1. Beta

In [5]:
crsp = pd.read_feather("raw_data/crsp_beta.ftr")

In [ ]:
# sort variables by permno and date
crsp = crsp.sort_values(by=['permno', 'date'])

# change variable format to int
crsp['permno'] = crsp['permno'].astype(int)

# Line up date to be end of month
crsp['date'] = pd.to_datetime(crsp['date'])

# add delisting return
dlret = pd.read_feather("dlret.ftr")

dlret.permno = dlret.permno.astype(int)
dlret['dlstdt'] = pd.to_datetime(dlret['dlstdt'])
dlret['date'] = dlret['dlstdt']

# merge delisting return to crsp return
crsp = pd.merge(crsp, dlret, how='left', on=['permno', 'date'])
crsp['dlret'] = crsp['dlret'].fillna(0)
crsp['ret'] = crsp['ret'].fillna(0)
crsp['retadj'] = (1 + crsp['ret']) * (1 + crsp['dlret']) - 1
crsp['exret'] = crsp['retadj'] - crsp['rf']

In [8]:
# find the closest trading day to the end of the month
crsp['monthend'] = crsp['date'] + MonthEnd(0)
crsp['date_diff'] = crsp['monthend'] - crsp['date']
date_temp = crsp.groupby(['permno', 'monthend'])['date_diff'].min()
date_temp = pd.DataFrame(date_temp)  # convert Series to DataFrame
date_temp.reset_index(inplace=True)
date_temp.rename(columns={'date_diff': 'min_diff'}, inplace=True)
crsp = pd.merge(crsp, date_temp, how='left', on=['permno', 'monthend'])
crsp['sig'] = np.where(crsp['date_diff'] == crsp['min_diff'], 1, np.nan)

# label every date of month end
crsp['month_count'] = crsp[crsp['sig'] == 1].groupby(['permno']).cumcount()

# label numbers of months for a firm
month_num = crsp[crsp['sig'] == 1].groupby(['permno'])['month_count'].tail(1)
month_num = month_num.astype(int)
month_num = month_num.reset_index(drop=True)

# mark the number of each month to each day of this month
crsp['month_count'] = crsp.groupby(['permno'])['month_count'].fillna(method='bfill')

# crate a firm list
df_firm = crsp.drop_duplicates(['permno'])
df_firm = df_firm[['permno']]
df_firm['permno'] = df_firm['permno'].astype(int)
df_firm = df_firm.reset_index(drop=True)
df_firm = df_firm.reset_index()
df_firm = df_firm.rename(columns={'index': 'count'})
df_firm['month_num'] = month_num

/var/folders/09/pqd87cvj3kjbrvbzxrht87d80000gn/T/ipykernel_95173/87601466.py:20: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  crsp['month_count'] = crsp.groupby(['permno'])['month_count'].fillna(method='bfill')
/var/folders/09/pqd87cvj3kjbrvbzxrht87d80000gn/T/ipykernel_95173/87601466.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  crsp['month_count'] = crsp.groupby(['permno'])['month_count'].fillna(method='bfill')


In [15]:
######################
# Calculate the beta #
######################
def get_beta(df, firm_list):
    """
    :param df: stock dataframe
    :param firm_list: list of firms matching stock dataframe
    :return: dataframe with variance of residual
    """
    half_life = 1.5  # 半衰期设为一个半月(rolling window为三个月)
    for firm, count, prog in zip(firm_list['permno'], firm_list['month_num'], range(firm_list['permno'].count() + 1)):
        prog = prog + 1
        print('processing permno %s' % firm, '/', 'finished', '%.2f%%' % ((prog / firm_list['permno'].count()) * 100))
        for i in range(count + 1):
            # if you want to change the rolling window, please change here: i - 2 means 3 months is a window.
            temp = df[(df['permno'] == firm) & (i - 2 <= df['month_count']) & (df['month_count'] <= i)]
            # if observations in last 3 months are less 21, we drop the rvar of this month
            if temp['permno'].count() < 21:
                continue
            else:
                rolling_window = temp['permno'].count()
                index = temp.tail(1).index
                X = temp[['mktrf']]
                Y = temp['exret']

                # 计算指数衰减权重
                time_differences = np.arange(rolling_window)[::-1]  # 从当前时间点往前的时间差
                weights = 0.5 ** (time_differences / half_life)

                # 添加常数项
                X = sm.add_constant(X)

                # 加权最小二乘回归
                model = sm.WLS(Y, X, weights=weights)
                results = model.fit()

                # beta
                beta = results.params['mktrf']
                df.loc[index, 'beta'] = beta
    return df


def sub_df(start, end, step):
    """
    :param start: the quantile to start cutting, usually it should be 0
    :param end: the quantile to end cutting, usually it should be 1
    :param step: quantile step
    :return: a dictionary including all the 'firm_list' dataframe and 'newly grouped stock data' dataframe
    """
    # 按照原来的逻辑先分10组
    temp = {}
    for i, h in zip(np.arange(start, end, step), range(int((end - start) / step))):
        print('processing splitting dataframe:', round(i, 2), 'to', round(i + step, 2))
        if i == 0:  # to get the left point
            temp['firm' + str(h)] = df_firm[df_firm['month_count'] <= df_firm['month_count'].quantile(i + step)]
            temp['crsp' + str(h)] = pd.merge(crsp, temp['firm' + str(h)], how='left',
                                             on='permno').dropna(subset=['month_count'])
        else:
            temp['firm' + str(h)] = df_firm[(df_firm['month_count'].quantile(i) < df_firm['month_count']) & (
                    df_firm['month_count'] <= df_firm['month_count'].quantile(i + step))]
            temp['crsp' + str(h)] = pd.merge(crsp, temp['firm' + str(h)], how='left',
                                             on='permno').dropna(subset=['month_count'])

    # 初始化新的10组df
    new_groups = {f'new_group_{i}': pd.DataFrame() for i in range(10)}

    # 将原来的10组数据均匀地分配到新的10组中，可以确保每个df的计算进度相近
    for key in temp:
        if key.startswith('crsp'):
            group_data = temp[key]
            for i, row in group_data.iterrows():
                new_group_index = len(new_groups[f'new_group_{i % 10}'])
                new_groups[f'new_group_{i % 10}'] = new_groups[f'new_group_{i % 10}'].append(row, ignore_index=True)

    return new_groups


def process(start, end, step):
    """

    :param start: the quantile to start cutting, usually it should be 0
    :param end: the quantile to end cutting, usually it should be 1
    :param step: quantile step
    :return: a dataframe with calculated variance of residual
    """
    df = sub_df(start, end, step)
    pool = mp.Pool()
    p_dict = {}
    for i in range(int((end-start)/step)):
        p_dict['p' + str(i)] = pool.apply_async(get_beta, (df['crsp%s' % i], df['firm%s' % i],))
    pool.close()
    pool.join()
    result = pd.DataFrame()
    print('processing pd.concat')
    for h in range(int((end-start)/step)):
        result = pd.concat([result, p_dict['p%s' % h].get()])
    return result


# calculate variance of residual through rolling window
# Note: please split dataframe according to your CPU situation. For example, we split dataframe to (1-0)/0.05 = 20 sub
# dataframes here, so the function will use 20 cores to calculate variance of residual.
crsp = process(0, 1, 0.125)

processing splitting dataframe: 0.0 to 0.12
processing splitting dataframe: 0.12 to 0.25
processing splitting dataframe: 0.25 to 0.38
processing splitting dataframe: 0.38 to 0.5
processing splitting dataframe: 0.5 to 0.62
processing splitting dataframe: 0.62 to 0.75
processing splitting dataframe: 0.75 to 0.88
processing splitting dataframe: 0.88 to 1.0
Processing chunk 1/8
processing permno 10001 / finished 0.05%
processing permno 10025 / finished 0.11%
processing permno 10026 / finished 0.16%
processing permno 10028 / finished 0.22%
processing permno 10032 / finished 0.27%
processing permno 10044 / finished 0.32%
processing permno 10051 / finished 0.38%
processing permno 10065 / finished 0.43%
processing permno 10066 / finished 0.48%
processing permno 10104 / finished 0.54%
processing permno 10107 / finished 0.59%
processing permno 10113 / finished 0.65%
processing permno 10116 / finished 0.70%
processing permno 10138 / finished 0.75%
processing permno 10145 / finished 0.81%
processi

In [19]:
# process dataframe
crsp = crsp.dropna(subset=['beta'])  # drop NA due to rolling
crsp = crsp.reset_index(drop=True)
crsp = crsp[['permno', 'date', 'beta']]
crsp.to_feather('processed_data/beta.ftr')